#Installing packages

In [ ]:
%%capture
!pip install nemo-toolkit['asr']
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode

## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

# Install TorchAudio
!pip install torchaudio>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

from omegaconf import OmegaConf
import os 

!mkdir conf 
!wget -P conf https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/speaker_tasks/recognition/conf/titanet-large.yaml
NEMO_ROOT = os.getcwd()
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/titanet-large.yaml')
config = OmegaConf.load(MODEL_CONFIG)

#Extracting Voiceprints

In [ ]:
import nemo.collections.asr as nemo_asr
from collections import defaultdict

speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')

[NeMo W 2023-03-15 14:50:41 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-03-15 14:50:45 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[NeMo I 2023-03-15 14:50:46 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/titanet_large/versions/v1/files/titanet-l.nemo to /root/.cache/torch/NeMo/NeMo_1.17.0rc0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2023-03-15 14:50:50 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-03-15 14:50:51 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-03-15 14:50:51 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-03-15 14:50:51 features:287] PADDING: 16
[NeMo I 2023-03-15 14:50:59 save_restore_connector:247] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.17.0rc0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.


In [ ]:
from tqdm import tqdm

In [ ]:
data_dir = "/content/drive/MyDrive/Colab Notebooks/MA LangTech/Research and Development/Emotional Speech Dataset (ESD)"

voiceprints = {"en": dict(), "ch": dict()}

for speaker in sorted(next(os.walk(data_dir))[1]):
  print(f"Processing speaker {speaker}")
  if 1 <= int(speaker) <= 10:
    lang = "ch"
  elif 11 <= int(speaker) <= 20:
    lang = "en"
  else:
    raise ValueError(f"Directory {speaker} does not exist")
  speaker_dir = os.path.join(data_dir, speaker)
  speaker_dict = dict()
  for emotion in next(os.walk(speaker_dir))[1]:
    emotion_dir = os.path.join(speaker_dir, emotion)
    emotion_dict = dict()
    for splt in next(os.walk(emotion_dir))[1]:
      splt_path = os.path.join(emotion_dir, splt)
      splt_list = []
      for sound_file in tqdm(list(filter(lambda x: ")" not in x, next(os.walk(splt_path))[2])),
                             desc=f"Emotion: {emotion:<7} split: {splt:<10}"): # REMOVE SPLICE LATER
        sound_path = os.path.join(splt_path, sound_file)
        emb = speaker_model.get_embedding(sound_path).cpu()
        splt_list.append(emb)
      emotion_dict[splt] = splt_list
    speaker_dict[emotion] = emotion_dict
  voiceprints[lang][speaker] = speaker_dict

Processing speaker 0001


Emotion: Neutral split: evaluation:   0%|          | 0/20 [00:00<?, ?it/s][NeMo W 2023-03-15 14:51:14 nemo_logging:349] /usr/local/lib/python3.9/dist-packages/nemo/collections/asr/models/label_models.py:432: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
      torch.tensor([audio], device=device),
    
Emotion: Happy   split: evaluation: 100%|██████████| 20/20 [00:05<00:00,  3.98it/s]


Processing speaker 0002


Emotion: Angry   split: train     : 100%|██████████| 300/300 [00:08<00:00, 33.87it/s]


Processing speaker 0003


...

Processing speaker 0004


Emotion: Sad     split: test      : 100%|██████████| 30/30 [00:10<00:00,  2.78it/s]


Processing speaker 0005


Emotion: Sad     split: evaluation:  95%|█████████▌| 20/21 [00:10<00:00,  1.75it/s][NeMo W 2023-03-15 15:00:21 nemo_logging:349] /usr/local/lib/python3.9/dist-packages/nemo/collections/asr/models/label_models.py:424: UserWarning: PySoundFile failed. Trying audioread instead.
      audio, sr = librosa.load(path2audio_file, sr=None)
    
[NeMo W 2023-03-15 15:00:21 nemo_logging:349] /usr/local/lib/python3.9/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
    	Deprecated as of librosa version 0.10.0.
    	It will be removed in librosa version 1.0.
      y, sr_native = __audioread_load(path, offset, duration, dtype)
    
Emotion: Sad     split: evaluation:  95%|█████████▌| 20/21 [00:10<00:00,  1.83it/s]


NoBackendError: ignored

In [ ]:
next(os.path.join(data_dir, "Angry"))

In [ ]:
sorted(next(os.walk(data_dir))[1])[-1:]

['0020']

In [ ]:
20*4*350

28000

In [ ]:
total_len = 0
for lang, lang_value in voiceprints.items():
  for speaker, speaker_value in lang_value.items():
    for emotion, emotion_value in speaker_value.items():
      for split, split_data in emotion_value.items():
        total_len += len(split_data)

print(total_len)

28005


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/masters_thesis/data/emotion_voiceprints

/content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/masters_thesis/data/emotion_voiceprints


In [ ]:
import pickle as pkl

save_dir = "/content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/masters_thesis/data/emotion_voiceprints"

with open(os.path.join(save_dir, "emotion_voiceprints.pkl"), "wb") as f:
  pkl.dump(voiceprints, f)

In [ ]:
%%capture
!pip install transformers

In [ ]:
import os
import pickle as pkl

In [ ]:
data_dir = "/content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/masters_thesis/data/emotion_voiceprints"

labels = {"happy": 0, "angry": 1, "sad": 2, "neutral": 3}

with open(os.path.join(data_dir, "emotion_voiceprints.pkl"), "rb") as f:
  voiceprints = pkl.load(f)
samples = []

langs = ["en", "ch"]

for lang, lang_value in voiceprints.items():
  if lang not in langs:
    continue
  for speaker, speaker_value in lang_value.items():
    for emotion, emotion_value in speaker_value.items():
      for split, split_data in emotion_value.items():
        for emb in split_data:
          samples.append((emb, labels[emotion.lower()]))

In [ ]:
len(samples)

28000

In [ ]:
pipeline.train_dataset.shuffle()
s = [i[1] for i in pipeline.train_dataset.samples]
print(s.index(0), s.index(1), s.index(2), s.index(3))

0 6 2 4


In [ ]:
random.shuffle(b)

In [ ]:
"en" in pipeline.train_dataset.LANGS

True

In [ ]:
b

[0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1]

#TitaNet SER

## Extract voiceprints

In [ ]:
import os
import pickle as pkl
vp_path = "/content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/masters_thesis/data/emotion_voiceprints"
voiceprints_path = os.path.join(vp_path, "emotion_voiceprints.pkl")

langs = ["en", "ch"]
labels = {"happy": 0, "angry": 1, "sad": 2, "neutral": 3}
eval_speakers = [7, 8]

with open(os.path.join(voiceprints_path), "rb") as f:
  voiceprints = pkl.load(f)
samples = {"en": dict(), "ch": dict()}

for eval_speaker in eval_speakers:
  for lang, lang_value in voiceprints.items():
    if lang not in langs:
      continue
    lang_dict = {"train": [], "evaluation": [], "test": []}
    for speaker, speaker_value in lang_value.items():
      mod_speaker = (int(speaker)-1)%10 + 1
      if mod_speaker >= 9:
        split_speaker = "test"
      elif mod_speaker == eval_speaker: # 7 or 8
        split_speaker = "evaluation"
      else:
        split_speaker = "train"
      for emotion, emotion_value in speaker_value.items():
        for split, split_value in emotion_value.items():
          for emb in split_value:
            lang_dict[split_speaker].append((emb, labels[emotion.lower()]))
    # samples[lang] = lang_dict
    save_path = os.path.join(vp_path, f"emotion_voiceprints_{lang}_{eval_speaker}.pkl")
    with open(save_path, "wb") as outfile:
      pkl.dump(lang_dict, outfile)

## Set up model

In [ ]:
def running_time(start, end):
  duration = end-start
  secs = int(duration%60)
  duration = duration//60
  mins = int(duration%60)
  hours = int(duration//60)

  return f"{str(hours).zfill(2)}:{str(mins).zfill(2)}:{str(secs).zfill(2)}"

In [ ]:
import torch
import numpy as np
import os
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pickle
import time
import random
import argparse
import pickle as pkl
from torch.optim.lr_scheduler import LambdaLR, ReduceLROnPlateau


class RawDataset(Dataset):
  """class to load the data

  X_train
    contains training data
  X_test
    contains training data
  y
    sentiment scores
    order is the same for training and test, so only one array created
  """
  def __init__(self, data_dir, langs, split, proportion=1.0):
    self.data_dir = data_dir
    self.LANGS = langs
    self.split = split
    self.proportion = proportion
    self.labels = {"happy": 0, "angry": 1, "sad": 2, "neutral": 3}
    self.idx_to_labels = ["happy", "angry", "sad", "neutral"]
    self.samples = self.get_dataset(self.data_dir, langs, split, proportion)


  def __len__(self):
    if ("ch" in self.LANGS and "en" in self.LANGS) and self.split != "test":
      return len(self.samples) // 2
    return len(self.samples)


  def shuffle(self):
    quarter_len = len(self.samples)
    english = self.samples[:quarter_len] + self.samples[2*quarter_len:3*quarter_len]
    random.shuffle(english)
    chinese = self.samples[quarter_len:2*quarter_len] + self.samples[3*quarter_len:]
    random.shuffle(chinese)
    self.samples = english[:quarter_len] + chinese[:quarter_len] \
                   + english[quarter_len:] + chinese[quarter_len:]



  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    X = self.samples[idx][0]
    y = self.samples[idx][1]
    return (X, y)


  def fetch_sound_text_mapping(self, data_dir, langs=["en"], split="train", proportion=1.0):
    """
    inputs:
      - data_dir: string; directory from which to extract wav-file paths
              |_ readme
              |_ speakers
                        |_ transcriptions
                        |_ emotions
                                  |_ splits

      - langs: list of strings; which languages are extracted
      - split: string; train, test, or evaluation
      - proportion: float; what proportion of files to extract 
                    (intended for development, set to 1.0 at implementation)

    outputs:
      - list: [float speech array, str emotion label]
    """

    samples = []

    # for lang in langs:
      # with open(os.path.join(data_dir, f"emotion_voiceprints_{lang}_7.pkl"), "rb") as f:
      #   voiceprints = pkl.load(f)
      # samples.extend(voiceprints[split])
    with open(os.path.join(data_dir, f"emotion_voiceprints.pkl"), "rb") as f:
      voiceprints = pkl.load(f)
    for lang, lang_value in voiceprints.items():
      if lang not in langs:
        continue
      for speaker, speaker_value in lang_value.items():
        for emotion, emotion_value in speaker_value.items():
          split_data = emotion_value[split]
          for emb in split_data:
            samples.append((emb, self.labels[emotion.lower()]))

    return samples

  
  def get_dataset(self, data_dir, langs, split, proportion=1.0):
    print(f"Getting {int(100*proportion)}% of samples")
    start = time.time()
    samples = self.fetch_sound_text_mapping(data_dir, langs, split, proportion)
    end = time.time()
    duration = end-start
    secs = int(duration%60)
    duration = duration//60
    mins = int(duration%60)
    hours = int(duration//60)

    if "en" in langs and "ch" in langs:
      quarter_len = len(samples)//4
      samples = samples[:quarter_len] + samples[2*quarter_len:3*quarter_len] \
                + samples[quarter_len:2*quarter_len] + samples[3*quarter_len:]

    print(f"Processing {int(100*proportion)}% of samples took {str(hours).zfill(2)}:{str(mins).zfill(2)}:{str(secs).zfill(2)}")

    return samples


class SERwav2vec2(nn.Module):
  def __init__(self, learning_rate, hidden_size=1024):
    super(SERwav2vec2, self).__init__()
    self.drop = 0.50
    self._clf = nn.Sequential(
                              nn.Linear(192, hidden_size),
                              nn.ReLU(),
                              nn.Dropout(self.drop),
                              nn.Linear(hidden_size, hidden_size),
                              nn.ReLU(),
                              nn.Dropout(self.drop),
                              nn.Linear(hidden_size, hidden_size),
                              nn.ReLU(),
                              nn.Dropout(self.drop),
                              nn.Linear(hidden_size, 4),
                              # nn.Linear(192, 4),
                              nn.LogSoftmax(dim=1))

    self.training_loss_ = list()                                                # For plotting
    self.training_accuracy_ = list()
    self.validation_loss_ = list()                                              # For plotting
    self.validation_accuracy_ = list()

    self.loss_function = nn.NLLLoss()
    self.val_loss_function = nn.NLLLoss(reduction="none")
    self.optimizer = torch.optim.Adam(self._clf.parameters(), lr=learning_rate,
                                      weight_decay=0.000)

    # lambda1 = lambda epoch: epoch // 30
    lambda2 = lambda epoch: 0.95 ** (epoch // 30)
    self.scheduler = LambdaLR(self.optimizer, lr_lambda=[lambda2])
    # self.scheduler = ReduceLROnPlateau(self.optimizer, 'min')#, factor=0.5,#, patience=0,
                                      #  threshold=1e-2, cooldown=0)

    self.test_perf = {"en": {"labels correct": {0: [], 1: [], 2: [], 3: []},
                             "test acc": 0, "test loss": 0},
                      "ch": {"labels correct": {0: [], 1: [], 2: [], 3: []},
                             "test acc": 0, "test loss": 0}
    }

    self.BATCH_SIZE = 20
    # self.BATCH_SIZE = 2

    if torch.cuda.is_available():                                               # Move the model to the GPU (if we have one)
      self.cuda()


  def coll_fn(self, X):
    max_len = max(input.shape[1] for input, target in X)
    num_tensors = len(X)
    padded = torch.zeros(num_tensors, max_len, dtype=torch.float32)
    for i, (input, output) in enumerate(X):
      padded[i, :input.shape[1]] = input
    labels = torch.tensor([label for input, label in X])
    return padded, labels


  def forward(self, X):
    """The forward pass through the network"""
    scores = self._clf(X)
    return scores

  def fit(self, train_set, epochs, validation_set=None):
    """fitting the model to the training data"""

    for epoch in range(epochs):                                                 # Times to loop over the full dataset
      if "en" in train_set.LANGS and "ch" in train_set.LANGS:
        train_set.shuffle()

      self.train()

      with tqdm(DataLoader(train_set, batch_size=self.BATCH_SIZE, shuffle=True,
                          num_workers=0, collate_fn=self.coll_fn),
                total=len(train_set)//self.BATCH_SIZE, unit="batch", 
                desc=f"Epoch {epoch+1}") as batches:
        losses = []
        accuracies = []
        for inputs, targets in batches:                                         # Loop once over the training data
          self.optimizer.zero_grad()                                            # Reset gradients
          if torch.cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda()
          
          scores = self(inputs)                                                 # Forward pass, flatten output
          loss = self.loss_function(scores, targets)                            # Get loss     
          loss.backward()                                                       # Backpropagate the error
          self.optimizer.step()                                                 # Run the optimizer to change the weights w.r.t the loss
          
          predictions = scores.argmax(dim=1)                                    # Calculate the batch training accuracy          
          correct = (predictions == targets).sum().item()                       # sum where prediction == target
          accuracy = correct / len(targets)*100
          self.training_accuracy_.append(accuracy)                              # Save the accuracy for plotting
          self.training_loss_.append(loss.item())                               # Save the loss for plotting
          losses.append(loss.item())
          accuracies.append(accuracy)
          batches.set_postfix(loss=sum(losses)/len(losses), accuracy=sum(accuracies)/len(accuracies))              # Update the progress bar
      # self.scheduler.step()                                                     # Run the optimizer to change the weights w.r.t the loss
          
      if validation_set:
        self.eval()
        with torch.no_grad():                                                       # Do not use the following forward passes to calculate a gradient
          n_correct = 0
          n_total = len(validation_set)
          losses = list()

          if "en" in train_set.LANGS and "ch" in train_set.LANGS:
            validation_set.shuffle()
          for inputs, targets in DataLoader(validation_set, batch_size=self.BATCH_SIZE, 
                                            shuffle=True, num_workers=0, collate_fn=self.coll_fn):           # Loop once over the test data
            if torch.cuda.is_available():
              inputs = inputs.cuda()
              targets = targets.cuda()
            scores = self(inputs)
            predictions = scores.argmax(dim=1)                                      # Runs the test data through the model and finds the predictions
            n_correct += (predictions == targets).sum().item()              # Sums the number of correct predictions
            losses += self.val_loss_function(scores, targets).tolist()
        loss = sum(losses)/n_total
        accuracy = 100*n_correct/n_total
        self.validation_loss_.append(loss)
        self.validation_accuracy_.append(accuracy)
        print(f"accuracy={accuracy}, loss={loss}")
      # self.scheduler.step(loss)

  def score(self, test_set):
    """returns a score given some data"""
    self.eval()
    labels_correct = {0: [], 1: [], 2: [], 3: []}
    lang = test_set.LANGS[0]
    with torch.no_grad():                                                       # Do not use the following forward passes to calculate a gradient
      n_correct = 0
      n_total = len(test_set)
      losses = list()
      for inputs, targets in DataLoader(test_set, batch_size=self.BATCH_SIZE, 
                                        shuffle=True, num_workers=0, collate_fn=self.coll_fn):             # Loop once over the test data
        if torch.cuda.is_available():
          inputs = inputs.cuda()
          targets = targets.cuda()
        scores = self(inputs)
        predictions = scores.argmax(dim=1)                                    # Runs the test data through the model and finds the predictions
        n_correct += (predictions == targets).sum().item()          # Sums the number of correct predictions
        for i, target in enumerate(targets):
          labels_correct[target.item()].append((target == predictions[i]).item())
        losses += self.val_loss_function(scores, targets).tolist()
    loss = sum(losses)/n_total
    accuracy = 100*n_correct/n_total
    # print(f"{lang} Test accuracy: {accuracy:.1f}, test loss: {loss:.1f}")
    self.test_perf[lang]["labels correct"] = labels_correct
    self.test_perf[lang]["test acc"] = accuracy
    self.test_perf[lang]["test loss"] = loss


class ModelDataPipeline():
  def __init__(self, epochs=10, learning_rate = 1e-4, langs=["en"], proportion=1.0):
    self.EPOCHS = epochs
    self.LEARNING_RATE = learning_rate
    self.LANGS = langs.copy()
    self.HIDDEN_SIZE = 1024

    print(f"Setting up epochs: {epochs}\n\tlearning rate: {learning_rate}\n\tlangs: {', '.join(langs)}")
    
    # check the input is valid
    for lang in self.LANGS:
      assert lang in ["en", "ch"], f"{lang} is not a valid language, should be one or both of 'en' or 'ch'"

    self.output_path = f"titanet_{'_'.join(self.LANGS)}_{self.EPOCHS}ep" #{self.MODEL_TYPE}_

    self.data_dir = "/content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/masters_thesis/data/emotion_voiceprints"
    # history_path = f"/home/mnachesa/rd_project/scripts/results/torch_history_{output_path}.pkl"
    self.perf_path = f"/content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/results/performance_{self.output_path}.pkl"
    self.saved_model_path = f"/content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/results/{self.output_path}.pth"

    self.train_dataset = RawDataset(self.data_dir, langs=self.LANGS, split="train", proportion=proportion)
    self.eval_dataset = RawDataset(self.data_dir, langs=self.LANGS, split="evaluation", proportion=proportion)
    self.en_test_dataset = RawDataset(self.data_dir, langs=["en"], split="test", proportion=proportion)
    self.ch_test_dataset = RawDataset(self.data_dir, langs=["ch"], split="test", proportion=proportion)
    self.sermod = SERwav2vec2(self.LEARNING_RATE, self.HIDDEN_SIZE)

    self.test_accs = {"ch": [], "en": []}
    self.test_losses = {"ch": [], "en": []}

  
  def train_model(self):
    print("Begin training")
    self.sermod.fit(self.train_dataset, epochs=self.EPOCHS, validation_set=self.eval_dataset)
    torch.save(self.sermod.state_dict(), self.saved_model_path)


  def test_model(self):
    self.sermod.score(self.en_test_dataset)
    self.sermod.score(self.ch_test_dataset)
    print(f"\nTest performance")
    print("lang accuracy loss")
    print("------------------")
    for lang in ["en", "ch"]:
      acc = self.sermod.test_perf[lang]["test acc"]
      loss = self.sermod.test_perf[lang]["test loss"]
      print(f"{lang:>4} {acc:>8.1f} {loss:>4.1f}")
    print()

  
  def save_model_performance(self):
    metrics = {"training accuracy": self.sermod.training_accuracy_,
               "training loss": self.sermod.training_loss_,
               "validation accuracy": self.sermod.validation_accuracy_,
               "validation loss": self.sermod.validation_loss_}
    test_perf = self.sermod.test_perf

    to_save = {"metrics": metrics, "test perf": test_perf}

    with open(self.perf_path, "wb") as f:
        pickle.dump(to_save, f)


  def set_data(self, eval_sp, langs, lang, remixed_data=True):
    if remixed_data:
      samples = dict() # {"ch": samples_ch, "en": samples_en}
      for train_lang in ["en", "ch"]:
        if train_lang == lang:
          with open(os.path.join(self.data_dir, f"emotion_voiceprints_{train_lang}_{eval_sp}.pkl"), "rb") as f:
            samples[train_lang] = pkl.load(f)
        else:
          with open(os.path.join(self.data_dir, f"emotion_voiceprints_{train_lang}_7.pkl"), "rb") as f:
            samples[train_lang] = pkl.load(f)
      
      self.train_dataset.samples = []
      self.eval_dataset.samples = []
      for lang in langs:
        self.train_dataset.samples += samples[lang]["train"]
        self.eval_dataset.samples += samples[lang]["evaluation"]
      self.ch_test_dataset.samples = samples["ch"]["test"]
      self.en_test_dataset.samples = samples["en"]["test"]

      if len(langs) == 2:
        quarter_len = len(self.train_dataset.samples)//4
        self.train_dataset.samples = self.train_dataset.samples[:quarter_len] \
                                    + self.train_dataset.samples[2*quarter_len:3*quarter_len] \
                                    + self.train_dataset.samples[quarter_len:2*quarter_len] \
                                    + self.train_dataset.samples[3*quarter_len:]


  def run_per_split(self, remixed_data=True):
    for lang in self.LANGS:
      for eval_sp in [7, 8]:
        self.sermod = SERwav2vec2(self.LEARNING_RATE, self.HIDDEN_SIZE)
        
        self.set_data(eval_sp, self.LANGS, lang, remixed_data)
        self.train_model()
        self.test_model()
        for test_lang in ["en", "ch"]:
          self.test_accs[test_lang].append(self.sermod.test_perf[test_lang]["test acc"])
          self.test_losses[test_lang].append(self.sermod.test_perf[test_lang]["test loss"])
        self.output_path = f"titanet_{'_'.join(self.LANGS)}_{self.EPOCHS}ep_eval_sp_{eval_sp}_{lang}_original_data_{not remixed_data}"
        self.perf_path = f"/content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/results/performance_{self.output_path}.pkl"
        self.save_model_performance()

    num_its = len(self.test_accs[lang])
    print(f"\n\nTrained on {' '.join(self.LANGS)} TitaNet voiceprints")
    print(f"Average test performance over {num_its} iteration(s)")
    print("lang accuracy loss")
    print("------------------")
    for lang in ["en", "ch"]:
      avg_acc = sum(self.test_accs[lang])/num_its
      avg_loss = sum(self.test_losses[lang])/num_its
      print(f"{lang:>4} {avg_acc:>8.1f} {avg_loss:>4.1f}")

if __name__ != "__main__":
  start = time.time()

  if False:
    ap = argparse.ArgumentParser()
    ap.add_argument('--epochs', default=10, type=int,
                      help='number of training epochs')
    ap.add_argument('--lr', default=1e-4, type=float,
                      help='model learning rate')
    ap.add_argument('--langs', default=["en"], nargs="+",
                      help='langs to train model on')
    ap.add_argument('--proportion', default=1.0, type=float,
                      help='Proportion of data to train on (mainly used in development)')

    args = ap.parse_args()

    epochs = args.epochs
    learning_rate = args.lr
    langs = args.langs
    proportion = args.proportion

  epochs = 10
  learning_rate = 1e-5
  langs = ["en", "ch"]
  proportion = 1.0

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               proportion=proportion)

  # pipeline.train_model()
  # pipeline.test_model()
  # pipeline.save_model_performance()

  end = time.time()
  duration = end-start
  secs = int(duration%60)
  duration = duration//60
  mins = int(duration%60)
  hours = int(duration//60)

  print(f"\nRunning the script took {str(hours).zfill(2)}:{str(mins).zfill(2)}:{str(secs).zfill(2)}")

In [ ]:
epochs = 40
learning_rate = 5e-5
proportion = 1.0

## Train and test models

### Train and test with new split

In [ ]:
if __name__ == "__main__":
  start = time.time()

  langs = ["en", "ch"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               proportion=proportion)
  
  pipeline.run_per_split()

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up epochs: 40
	learning rate: 5e-05
	langs: en, ch
Getting 100% of samples
Processing 100% of samples took 00:00:02
Getting 100% of samples
Processing 100% of samples took 00:00:02
Getting 100% of samples
Processing 100% of samples took 00:00:03
Getting 100% of samples
Processing 100% of samples took 00:00:02
Begin training


Epoch 1: 100%|██████████| 490/490 [00:02<00:00, 165.68batch/s, accuracy=33.4, loss=1.37]


accuracy=63.285714285714285, loss=1.3089099437849863


Epoch 2: 100%|██████████| 490/490 [00:03<00:00, 158.79batch/s, accuracy=61.6, loss=0.909]


accuracy=72.28571428571429, loss=0.7086757837608456


...

accuracy=57.5, loss=1.834394583341528


Epoch 39: 100%|██████████| 490/490 [00:03<00:00, 147.83batch/s, accuracy=98.3, loss=0.0455]


accuracy=59.357142857142854, loss=1.6764136591380874


Epoch 40: 100%|██████████| 490/490 [00:03<00:00, 152.95batch/s, accuracy=98.3, loss=0.0479]


accuracy=56.92857142857143, loss=1.7695332314712866

Test performance
lang accuracy loss
------------------
  en     63.8  1.4
  ch     76.4  0.8



Trained on en ch TitaNet voiceprints
Average test performance over 4 iteration(s)
lang accuracy loss
------------------
  en     62.7  1.5
  ch     82.1  0.6

Running the script took 00:09:02


In [ ]:
if __name__ == "__main__":
  start = time.time()

  langs = ["en"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               proportion=proportion)
  
  pipeline.run_per_split()

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up epochs: 40
	learning rate: 5e-05
	langs: en
Getting 100% of samples
Processing 100% of samples took 00:00:04
Getting 100% of samples
Processing 100% of samples took 00:00:03
Getting 100% of samples
Processing 100% of samples took 00:00:03
Getting 100% of samples
Processing 100% of samples took 00:00:02
Begin training


Epoch 1: 100%|██████████| 490/490 [00:03<00:00, 158.87batch/s, accuracy=32, loss=1.38]


accuracy=51.142857142857146, loss=1.335924832054547


Epoch 2: 100%|██████████| 490/490 [00:03<00:00, 161.19batch/s, accuracy=59.4, loss=0.954]


accuracy=54.857142857142854, loss=0.9165385457447597


...

accuracy=63.57142857142857, loss=1.9565556357458334


Epoch 39: 100%|██████████| 490/490 [00:03<00:00, 154.29batch/s, accuracy=99, loss=0.0293]


accuracy=64.78571428571429, loss=1.8722288984996187


Epoch 40: 100%|██████████| 490/490 [00:03<00:00, 151.62batch/s, accuracy=99, loss=0.0281]


accuracy=65.28571428571429, loss=1.800681852523078

Test performance
lang accuracy loss
------------------
  en     61.3  1.8
  ch     60.4  1.9



Trained on en TitaNet voiceprints
Average test performance over 2 iteration(s)
lang accuracy loss
------------------
  en     61.1  1.8
  ch     58.8  2.1

Running the script took 00:04:38


In [ ]:
if __name__ == "__main__":
  start = time.time()

  langs = ["ch"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               proportion=proportion)
  
  pipeline.run_per_split()

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up epochs: 40
	learning rate: 5e-05
	langs: ch
Getting 100% of samples
Processing 100% of samples took 00:00:03
Getting 100% of samples
Processing 100% of samples took 00:00:02
Getting 100% of samples
Processing 100% of samples took 00:00:02
Getting 100% of samples
Processing 100% of samples took 00:00:02
Begin training


Epoch 1: 100%|██████████| 490/490 [00:03<00:00, 152.12batch/s, accuracy=40.8, loss=1.31]


accuracy=38.07142857142857, loss=1.0702404619540486


Epoch 2: 100%|██████████| 490/490 [00:03<00:00, 155.15batch/s, accuracy=82.8, loss=0.509]


accuracy=47.857142857142854, loss=1.2050208873172024


...

accuracy=40.642857142857146, loss=4.191871324363705


Epoch 39: 100%|██████████| 490/490 [00:03<00:00, 152.30batch/s, accuracy=99.9, loss=0.00425]


accuracy=52.07142857142857, loss=2.851654743151373


Epoch 40: 100%|██████████| 490/490 [00:03<00:00, 158.07batch/s, accuracy=99.9, loss=0.00371]


accuracy=45.214285714285715, loss=3.616266813122932

Test performance
lang accuracy loss
------------------
  en     46.6  3.8
  ch     76.5  1.3



Trained on ch TitaNet voiceprints
Average test performance over 2 iteration(s)
lang accuracy loss
------------------
  en     47.4  4.2
  ch     79.4  1.1

Running the script took 00:04:39


### Train and test with original split

In [ ]:
if __name__ == "__main__":
  start = time.time()

  langs = ["en", "ch"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               proportion=proportion)
  
  pipeline.run_per_split(remixed_data=False)

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up epochs: 40
	learning rate: 5e-05
	langs: en, ch
Getting 100% of samples
Processing 100% of samples took 00:00:02
Getting 100% of samples
Processing 100% of samples took 00:00:02
Getting 100% of samples
Processing 100% of samples took 00:00:02
Getting 100% of samples
Processing 100% of samples took 00:00:02
Begin training


Epoch 1: 100%|██████████| 600/600 [00:03<00:00, 155.58batch/s, accuracy=33.8, loss=1.37]


accuracy=57.625, loss=1.1994858995079993


Epoch 2: 100%|██████████| 600/600 [00:03<00:00, 154.93batch/s, accuracy=63.6, loss=0.858]


accuracy=71.375, loss=0.6762197655299679


...

accuracy=95.25, loss=0.13404950059037113


Epoch 39: 100%|██████████| 600/600 [00:04<00:00, 149.23batch/s, accuracy=98.7, loss=0.0407]


accuracy=94.625, loss=0.1416468990430959


Epoch 40: 100%|██████████| 600/600 [00:03<00:00, 161.70batch/s, accuracy=98.6, loss=0.038]


accuracy=94.75, loss=0.1417471871375083

Test performance
lang accuracy loss
------------------
  en     95.2  0.1
  ch     98.5  0.1



Trained on en ch TitaNet voiceprints
Average test performance over 4 iteration(s)
lang accuracy loss
------------------
  en     96.0  0.1
  ch     98.6  0.1

Running the script took 00:10:33


In [ ]:
if __name__ == "__main__":
  start = time.time()

  langs = ["en"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               proportion=proportion)
  
  pipeline.run_per_split(remixed_data=False)

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up epochs: 40
	learning rate: 5e-05
	langs: en
Getting 100% of samples
Processing 100% of samples took 00:00:03
Getting 100% of samples
Processing 100% of samples took 00:00:04
Getting 100% of samples
Processing 100% of samples took 00:00:03
Getting 100% of samples
Processing 100% of samples took 00:00:02
Begin training


Epoch 1: 100%|██████████| 600/600 [00:04<00:00, 148.14batch/s, accuracy=34.5, loss=1.36]


accuracy=54.875, loss=1.1414355506747962


Epoch 2: 100%|██████████| 600/600 [00:03<00:00, 152.88batch/s, accuracy=64.8, loss=0.824]


accuracy=68.25, loss=0.7274786843464244


...

accuracy=93.5, loss=0.23124920807972274


Epoch 39: 100%|██████████| 600/600 [00:03<00:00, 150.60batch/s, accuracy=99.2, loss=0.0236]


accuracy=93.5, loss=0.24214516397805025


Epoch 40: 100%|██████████| 600/600 [00:03<00:00, 154.56batch/s, accuracy=99.2, loss=0.0234]


accuracy=93.5, loss=0.24199773317618628

Test performance
lang accuracy loss
------------------
  en     96.2  0.1
  ch     59.1  2.7



Trained on en TitaNet voiceprints
Average test performance over 2 iteration(s)
lang accuracy loss
------------------
  en     96.2  0.1
  ch     57.9  2.9

Running the script took 00:05:22


In [ ]:
if __name__ == "__main__":
  start = time.time()

  langs = ["ch"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               proportion=proportion)
  
  pipeline.run_per_split(remixed_data=False)

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up epochs: 40
	learning rate: 5e-05
	langs: ch
Getting 100% of samples
Processing 100% of samples took 00:00:02
Getting 100% of samples
Processing 100% of samples took 00:00:03
Getting 100% of samples
Processing 100% of samples took 00:00:02
Getting 100% of samples
Processing 100% of samples took 00:00:02
Begin training


Epoch 1: 100%|██████████| 600/600 [00:03<00:00, 160.74batch/s, accuracy=45.8, loss=1.22]


accuracy=69.75, loss=0.7453856534895021


Epoch 2: 100%|██████████| 600/600 [00:03<00:00, 159.15batch/s, accuracy=84.3, loss=0.449]


accuracy=88.625, loss=0.3271681316419563


...

accuracy=97.125, loss=0.10706411768097662


Epoch 39: 100%|██████████| 600/600 [00:03<00:00, 154.94batch/s, accuracy=99.8, loss=0.00482]


accuracy=97.0, loss=0.11542115234974147


Epoch 40: 100%|██████████| 600/600 [00:03<00:00, 152.13batch/s, accuracy=99.8, loss=0.00499]


accuracy=96.625, loss=0.11892583026399421

Test performance
lang accuracy loss
------------------
  en     47.7  4.6
  ch     98.3  0.1



Trained on ch TitaNet voiceprints
Average test performance over 2 iteration(s)
lang accuracy loss
------------------
  en     48.8  4.4
  ch     98.2  0.1

Running the script took 00:05:20


#TitaNet Voiceprint T-SNE

In [ ]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
import pickle as pkl
import os

class voiceprints:
  def __init__(self):
    self.data_dir = "/content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/masters_thesis/data/emotion_voiceprints"
    self.labels = {"happy": 0, "angry": 1, "sad": 2, "neutral": 3}
    self.langs = ["en", "ch"]
    self.samples = self.load_voiceprints()


  def load_voiceprints(self):
    with open(os.path.join(self.data_dir, "emotion_voiceprints.pkl"), "rb") as f:
      voiceprints = pkl.load(f)
    samples = []

    for lang, lang_value in voiceprints.items():
      if lang not in self.langs:
        continue
      for speaker, speaker_value in lang_value.items():
        for emotion, emotion_value in speaker_value.items():
          for split, split_value in emotion_value.items():
            for emb in split_value:
              samples.append((np.array(emb[0]), emotion.lower(), lang, speaker))
    return samples


class tsne_analysis:
  def __init__(self, samples, langs=["en", "ch"], perplexity=30):
    self.df = pd.DataFrame(samples, columns=["emb", "emotion", "lang", "speaker"])
    self.df = self.df[self.df.lang.apply(lambda x: x in langs)]
    self.langs = langs[:]
    self.tsne = TSNE(n_components=2, verbose=1, random_state=123, perplexity=perplexity)
    self.do_tsne()


  def do_tsne(self):
    z = self.tsne.fit_transform(np.array(self.df.emb.tolist()))
    self.df["comp-1"] = z[:,0]
    self.df["comp-2"] = z[:,1]

  
  def tsne_graph(self, parameters=["emotion"]):
    num_colors = 1
    for param in parameters:
      num_colors *= len(set(self.df[param].tolist()))
    hue = self.df[parameters].apply(lambda x: tuple([x[i] for i in range(len(x))]), axis=1).tolist()
    if len(hue[0]) == 1:
      hue = [i[0] for i in hue]
    ax = sns.scatterplot(x="comp-1", y="comp-2", 
                    hue=hue,
                    palette=sns.color_palette("hls", num_colors),
                    data=self.df)
    ax.set(title=f"embedding separation for 4 emotions in {' and '.join(self.langs)}")
    sns.move_legend(ax, "center left", bbox_to_anchor=(1,0.5))
    # plt.savefig(os.path.join(results_dir, f'T-SNE for all speeches at {d} speech length.png'), bbox_inches='tight')
    plt.show()
    plt.close()

voiceprints = voiceprints()

In [ ]:
en_ch_tsne = tsne_analysis(voiceprints.samples)
en_tsne = tsne_analysis(voiceprints.samples, langs=["en"])
ch_tsne = tsne_analysis(voiceprints.samples, langs=["ch"])

In [ ]:
en_ch_tsne.tsne_graph(["lang"])

In [ ]:
en_tsne.tsne_graph()

In [ ]:
ch_tsne.tsne_graph()

In [ ]:
ax = sns.scatterplot(x="comp-1", y="comp-2", hue=df.speaker.tolist(),
                palette=sns.color_palette("hls", 20),
                data=df)
ax.set(title=f"embedding separation for 4 emotions in English and Chinese")
sns.move_legend(ax, "center left", bbox_to_anchor=(1,0.5))
# plt.savefig(os.path.join(results_dir, f'T-SNE for all speeches at {d} speech length.png'), bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
ax = sns.scatterplot(x="comp-1", y="comp-2", hue=df.lang.tolist(),
                palette=sns.color_palette("hls", 2),
                data=df)
ax.set(title=f"embedding separation for 4 emotions in English and Chinese")
sns.move_legend(ax, "center left", bbox_to_anchor=(1,0.5))
# plt.savefig(os.path.join(results_dir, f'T-SNE for all speeches at {d} speech length.png'), bbox_inches='tight')
plt.show()
plt.close()

# Wav2vec2 SER

## Set up model

In [ ]:
%%capture
!pip install transformers

In [ ]:
# !pip install transformers

import soundfile as sf
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model, Wav2Vec2Config
import numpy as np
import os
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pickle
import time
import random
import argparse
from torch.optim.lr_scheduler import LambdaLR, ReduceLROnPlateau


class RawDataset(Dataset):
  """class to load the data

  X_train
    contains training data
  X_test
    contains training data
  y
    sentiment scores
    order is the same for training and test, so only one array created
  """
  def __init__(self, data_dir, langs, split, proportion=1.0):
    self.data_dir = data_dir
    self.LANGS = langs
    self.split = split
    self.proportion = proportion
    self.REQUIRED_SAMPLE_RATE = 16000
    self.AUDIO_MAXLEN = 246000
    self.labels = {"happy": 0, "angry": 1, "sad": 2, "neutral": 3}
    self.idx_to_labels = ["happy", "angry", "sad", "neutral"]
    self.processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    # self.X, self.y = self.get_dataset(self.data_dir, langs, split, proportion)
    self.samples = self.get_dataset(self.data_dir, langs, split, proportion)


  def __len__(self):
    # return len(self.X)
    if ("ch" in self.LANGS and "en" in self.LANGS) and self.split != "test":
      return len(self.samples) // 2
    return len(self.samples)

  
  def preprocess_speech(self, audio):
    audio = torch.tensor(audio, dtype=torch.float32)
    processed = self.processor(audio, return_tensors="pt", sampling_rate=self.REQUIRED_SAMPLE_RATE).input_values
    return processed
  

  def preprocess_text(self, label):
    return torch.tensor(self.labels[label], dtype=torch.long)


  def shuffle(self):
    quarter_len = len(self.samples)
    english = self.samples[:quarter_len] + self.samples[2*quarter_len:3*quarter_len]
    random.shuffle(english)
    chinese = self.samples[quarter_len:2*quarter_len] + self.samples[3*quarter_len:]
    random.shuffle(chinese)
    self.samples = english[:quarter_len] + chinese[:quarter_len] \
                   + english[quarter_len:] + chinese[quarter_len:]



  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    X = self.samples[idx][0]
    # X = self.preprocess_speech(X)
    y = self.samples[idx][1]
    # y = self.preprocess_text(y)
    return (X, y)


  def read_wav_file(self, file_path):
    audio, sample_rate = sf.read(file_path)
    if sample_rate != self.REQUIRED_SAMPLE_RATE:
        raise ValueError(
            f"sample rate (={sample_rate}) of your files must be {self.REQUIRED_SAMPLE_RATE}"
        )
    file_id = os.path.split(file_path)[-1][:-len(".wav")]
    return {file_id: audio}

  def print_box(self, text):
    line = len(text) * '-'
    print(f"+{line}+\n"+\
          f"|{text}|\n"+\
          f"+{line}+")

  def get_wav_file_paths(self, data_dir, langs, split, proportion):
    wav_files = []
    fr = 0 if "ch" in langs else 10
    to = 20 if "en" in langs else 10
    
    self.print_box(split)
    for speaker in sorted(next(os.walk(data_dir))[1])[fr:to]:
      print(f"Processing {speaker}")
      sp_path = os.path.join(data_dir, speaker)
      for emo_dir in next(os.walk(sp_path))[1]:
        print(f"\tProcessing {emo_dir}")
        emo_path = os.path.join(sp_path, emo_dir)
        split_path = os.path.join(emo_path, split)
        files = os.listdir(split_path)
        num_files = int(len(files)*proportion)
        for file in files[:num_files]:
            file_path = os.path.join(split_path, file)
            wav_files.append(file_path)
    return wav_files


  def get_labels(self, data_dir, langs, split, proportion):
    labels = {}
    fr = 0 if "ch" in langs else 10
    to = 20 if "en" in langs else 10
    
    self.print_box(split)
    for speaker in sorted(next(os.walk(data_dir))[1])[fr:to]:
      print(f"Processing {speaker}")
      sp_path = os.path.join(data_dir, speaker)
      for emo_dir in next(os.walk(sp_path))[1]:
        print(f"\tProcessing {emo_dir}")
        emo_path = os.path.join(sp_path, emo_dir)
        emo_dir = emo_dir.lower()
        split_path = os.path.join(emo_path, split)
        files = os.listdir(split_path)
        num_files = int(len(files)*proportion)
        for file in files[:num_files]:
            labels[file.strip(".wav")] = emo_dir
    return labels



  def fetch_sound_text_mapping(self, data_dir, langs=["en"], split="train", proportion=1.0):
    """
    inputs:
      - data_dir: string; directory from which to extract wav-file paths
              |_ readme
              |_ speakers
                        |_ transcriptions
                        |_ emotions
                                  |_ splits

      - langs: list of strings; which languages are extracted
      - split: string; train, test, or evaluation
      - proportion: float; what proportion of files to extract 
                    (intended for development, set to 1.0 at implementation)

    outputs:
      - list: [float speech array, str emotion label]
    """
    all_files = os.listdir(data_dir)

    print("Getting wav-file paths")
    wav_files = self.get_wav_file_paths(data_dir, langs, split, proportion)

    print("\nGetting labels")
    labels = self.get_labels(data_dir, langs, split, proportion)

    print("\nExtracting speech")
    speech_samples = {}
    speaker = ""
    emotion = ""
    for f in wav_files:
      # print(f)
      if f.split("/")[-4] != speaker:
        speaker = f.split("/")[-4]
        print(f"Processing {speaker}")
      if f.split("/")[-3] != emotion:
        emotion = f.split("/")[-3]
        print(f"\tProcessing {emotion}")
      speech_samples.update(self.read_wav_file(f))

    assert len(labels) == len(speech_samples)

    samples = [(speech_samples[file_id], labels[file_id]) for file_id in speech_samples.keys() if len(speech_samples[file_id]) <= self.AUDIO_MAXLEN]
    return samples

  
  def get_dataset(self, data_dir, langs, split, proportion=1.0):
    print(f"Getting {int(100*proportion)}% of samples")
    start = time.time()
    samples = self.fetch_sound_text_mapping(data_dir, langs, split, proportion)
    end = time.time()
    duration = end-start
    secs = int(duration%60)
    duration = duration//60
    mins = int(duration%60)
    hours = int(duration//60)

    if "en" in langs and "ch" in langs:
      quarter_len = len(samples)//4
      samples = samples[:quarter_len] + samples[2*quarter_len:3*quarter_len] \
                + samples[quarter_len:2*quarter_len] + samples[3*quarter_len:]

    print(f"Processing {int(100*proportion)}% of samples took {str(hours).zfill(2)}:{str(mins).zfill(2)}:{str(secs).zfill(2)}")
    
    # X = [i[0] for i in samples]
    # y = [i[1] for i in samples]
    # dataset = RawDataset(samples)

    return samples
    # return X, y


class SERwav2vec2(nn.Module):
  def __init__(self, pretrain_path, learning_rate, linear_size, hidden_size=1024):
    super(SERwav2vec2, self).__init__()
    self.drop = 0.5
    self._ser = Wav2Vec2Model.from_pretrained(pretrain_path)
    self._pooler = nn.Sequential(nn.AdaptiveAvgPool1d(1),
                              nn.Flatten(start_dim=1))
    self._clf = nn.Sequential(nn.Linear(linear_size, hidden_size),
                              nn.ReLU(),
                              nn.Dropout(self.drop),
                              nn.Linear(hidden_size, hidden_size),
                              nn.ReLU(),
                              nn.Dropout(self.drop),
                              nn.Linear(hidden_size, hidden_size),
                              nn.ReLU(),
                              nn.Dropout(self.drop),
                              nn.Linear(hidden_size, 4),
                              # nn.Linear(linear_size, 4),
                              nn.LogSoftmax(dim=1))

    self.training_loss_ = list()                                                # For plotting
    self.training_accuracy_ = list()
    self.validation_loss_ = list()                                                # For plotting
    self.validation_accuracy_ = list()

    self.loss_function = nn.NLLLoss()
    self.val_loss_function = nn.NLLLoss(reduction="none")
    self.optimizer = torch.optim.Adam(self._clf.parameters(), lr=learning_rate)

    lambda2 = lambda epoch: 0.95 ** (epoch // 30)
    self.scheduler = LambdaLR(self.optimizer, lr_lambda=[lambda2])

    self.test_perf = {"en": {"labels correct": {0: [], 1: [], 2: [], 3: []},
                             "test acc": 0, "test loss": 0},
                      "ch": {"labels correct": {0: [], 1: [], 2: [], 3: []},
                             "test acc": 0, "test loss": 0}
    }

    # self.labels_correct = {0: 0, 1: 0, 2: 0, 3: 0} # for testing
    # self.test_acc, self.test_loss = 0, 0

    self.BATCH_SIZE = 20

    if torch.cuda.is_available():                                               # Move the model to the GPU (if we have one)
      self.cuda()


  def coll_fn(self, X):
    max_len = max(input.shape[1] for input, target in X)
    num_tensors = len(X)
    padded = torch.zeros(num_tensors, max_len, dtype=torch.float32)
    for i, (input, output) in enumerate(X):
      padded[i, :input.shape[1]] = input
    labels = torch.tensor([label for input, label in X])
    return padded, labels


  def forward(self, X):
    """The forward pass through the network"""
    # print(X)
    # with torch.no_grad():
    #   hidden = self._ser(X).last_hidden_state
    # hidden = self._pooler(hidden.transpose(1,2))
    # scores = self._clf(hidden)
    scores = self._clf(X)
    # scores = self._clf(hidden)
    # del c
    return scores

  def fit(self, train_set, epochs, validation_set=None):
    """fitting the model to the training data"""

    for epoch in range(epochs):                                                 # Times to loop over the full dataset
      self.train()
      if "en" in train_set.LANGS and "ch" in train_set.LANGS:
        train_set.shuffle()

      with tqdm(DataLoader(train_set, batch_size=self.BATCH_SIZE, shuffle=True,
                          num_workers=0, collate_fn=self.coll_fn),
                total=len(train_set)//self.BATCH_SIZE, unit="batch", 
                desc=f"Epoch {epoch+1}") as batches:
        losses = []
        accuracies = []
        for inputs, targets in batches:                                         # Loop once over the training data
          self.optimizer.zero_grad()                                            # Reset gradients
          if torch.cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda()
          
          scores = self(inputs)                                                 # Forward pass, flatten output
          loss = self.loss_function(scores, targets)                            # Get loss 
          # print(scores, targets, loss)            
          loss.backward()                                                       # Backpropagate the error
          self.optimizer.step()                                                 # Run the optimizer to change the weights w.r.t the loss
          
          predictions = scores.argmax(dim=1)                                    # Calculate the batch training accuracy          
          correct = (predictions == targets).sum().item()                       # sum where prediction == target
          accuracy = correct / len(targets)*100
          self.training_accuracy_.append(accuracy)                              # Save the accuracy for plotting
          self.training_loss_.append(loss.item())                               # Save the loss for plotting
          losses.append(loss.item())
          accuracies.append(accuracy)
          # batches.set_postfix(loss=loss.item(), accuracy=accuracy)              # Update the progress bar
          batches.set_postfix(loss=sum(losses)/len(losses), accuracy=sum(accuracies)/len(accuracies))              # Update the progress bar
          # del tokenized, scores, targets, loss, predictions, correct, accuracy  # delete variables to save the GPU RAM
          # gc.collect()
          # torch.cuda.empty_cache()
      self.scheduler.step()

      if validation_set:
        self.eval()
        with torch.no_grad():                                                       # Do not use the following forward passes to calculate a gradient
          n_correct = 0
          n_total = len(validation_set)
          losses = list()

          if "en" in train_set.LANGS and "ch" in train_set.LANGS:
            validation_set.shuffle()
          for inputs, targets in DataLoader(validation_set, batch_size=self.BATCH_SIZE, 
                                            shuffle=True, num_workers=0, collate_fn=self.coll_fn):           # Loop once over the test data
            if torch.cuda.is_available():
              inputs = inputs.cuda()
              targets = targets.cuda()
            scores = self(inputs)
            predictions = scores.argmax(dim=1)                                      # Runs the test data through the model and finds the predictions
            n_correct += (predictions == targets).sum().item()              # Sums the number of correct predictions
            losses += self.val_loss_function(scores, targets).tolist()
        loss = sum(losses)/n_total
        accuracy = 100*n_correct/n_total
        self.validation_loss_.append(loss)
        self.validation_accuracy_.append(accuracy)
        print(f"accuracy={accuracy}, loss={loss}")

  def score(self, test_set):
    """returns a score given some data"""
    labels_correct = {0: [], 1: [], 2: [], 3: []}
    lang = test_set.LANGS[0]
    with torch.no_grad():                                                       # Do not use the following forward passes to calculate a gradient
      self.eval()
      n_correct = 0
      n_total = len(test_set)
      losses = list()
      for inputs, targets in DataLoader(test_set, batch_size=self.BATCH_SIZE, 
                                        shuffle=True, num_workers=0, collate_fn=self.coll_fn):             # Loop once over the test data
        if torch.cuda.is_available():
          inputs = inputs.cuda()
          targets = targets.cuda()
        scores = self(inputs)
        predictions = scores.argmax(dim=1)                                    # Runs the test data through the model and finds the predictions
        n_correct += (predictions == targets).sum().item()          # Sums the number of correct predictions
        for i, target in enumerate(targets):
          labels_correct[target.item()].append((target == predictions[i]).item())
        losses += self.val_loss_function(scores, targets).tolist()
    loss = sum(losses)/n_total
    accuracy = 100*n_correct/n_total
    # print(f"{lang} Test accuracy: {accuracy:.1f}, test loss: {loss:.1f}")
    self.test_perf[lang]["labels correct"] = labels_correct
    self.test_perf[lang]["test acc"] = accuracy
    self.test_perf[lang]["test loss"] = loss
    # self.test_acc, self.test_loss = accuracy, loss


class ModelDataPipeline():
  def __init__(self, epochs=10, learning_rate = 1e-4, langs=["en"], model_type="mono", proportion=1.0):
    # change these
    self.EPOCHS = epochs
    self.LEARNING_RATE = learning_rate
    self.LANGS = langs.copy()
    self.MODEL_TYPE = model_type

    print(f"Setting up {model_type} model with\n\tepochs: {epochs}\n\tlearning rate: {learning_rate}\n\tlangs: {', '.join(langs)}")
    
    # check the input is valid
    for lang in self.LANGS:
      assert lang in ["en", "ch"], f"{lang} is not a valid language, should be one or both of 'en' or 'ch'"
    assert self.MODEL_TYPE in ["mono", "xlsr"], f"{self.MODEL_TYPE} is not a valid model type, should be either 'mono' or xlsr'"
    if self.MODEL_TYPE == "mono":
      self.PRETRAIN_PATH = "facebook/wav2vec2-base"#-960h"
      self.LINEAR_SIZE = 768
    elif self.MODEL_TYPE == "xlsr":
      self.PRETRAIN_PATH = "facebook/wav2vec2-large-xlsr-53"
      self.LINEAR_SIZE = 1024

    self.output_path = f"{self.MODEL_TYPE}_{'_'.join(self.LANGS)}_{self.EPOCHS}ep"

    self.data_dir = "/content/drive/MyDrive/Colab Notebooks/MA LangTech/Research and Development/Emotional Speech Dataset (ESD)"
    self.pickle_dir = "/content/drive/MyDrive/Colab Notebooks/MA LangTech/Research and Development/pickle_datasets"
    # history_path = f"/home/mnachesa/rd_project/scripts/results/torch_history_{output_path}.pkl"
    self.perf_path = f"/content/drive/MyDrive/Colab Notebooks/MA LangTech/Research and Development/performance_{self.output_path}.pkl"
    self.saved_model_path = f"/content/drive/MyDrive/Colab Notebooks/MA LangTech/Research and Development/{self.output_path}.pth"

    # configuration = Wav2Vec2Config()
    # self.train_dataset = RawDataset(self.data_dir, langs=self.LANGS, split="train", proportion=proportion)
    # self.eval_dataset = RawDataset(self.data_dir, langs=self.LANGS, split="evaluation", proportion=proportion)
    # self.en_test_dataset = RawDataset(self.data_dir, langs=["en"], split="test", proportion=proportion)
    # self.ch_test_dataset = RawDataset(self.data_dir, langs=["ch"], split="test", proportion=proportion)
    self.sermod = SERwav2vec2(self.PRETRAIN_PATH, self.LEARNING_RATE, self.LINEAR_SIZE)
    # print(type(self.train_dataset))

    self.test_accs = {"ch": [], "en": []}
    self.test_losses = {"ch": [], "en": []}

  
  def train_model(self):
    print("Begin training")
    self.sermod.fit(self.train_dataset, epochs=self.EPOCHS, validation_set=self.eval_dataset)
    # torch.save(self.sermod.state_dict(), self.saved_model_path)


  def test_model(self):
    self.sermod.score(self.en_test_dataset)
    self.sermod.score(self.ch_test_dataset)
    print(f"\nTest performance")
    print("lang accuracy loss")
    print("------------------")
    for lang in ["en", "ch"]:
      acc = self.sermod.test_perf[lang]["test acc"]
      loss = self.sermod.test_perf[lang]["test loss"]
      print(f"{lang:>4} {acc:>8.1f} {loss:>4.1f}")
    print()

  
  def save_model_performance(self):
    metrics = {"training accuracy": self.sermod.training_accuracy_,
               "training loss": self.sermod.training_loss_,
               "validation accuracy": self.sermod.validation_accuracy_,
               "validation loss": self.sermod.validation_loss_}
    test_perf = self.sermod.test_perf

    to_save = {"metrics": metrics, "test perf": test_perf}

    with open(self.perf_path, "wb") as f:
        pickle.dump(to_save, f)

    
  def set_data(self, eval_sp, langs, lang, remixed_data=True):
    if remixed_data:
      eval_sp -= 1
      inp_eval_sp = eval_sp
      samples_en = {"train": [], "evaluation": [], "test": []}
      samples_ch = {"train": [], "evaluation": [], "test": []}
      samples = {"en": samples_en, "ch": samples_ch}

      input_samples = {"en": {}, "ch": {}}

      with open(os.path.join(self.pickle_dir, f"en_train_dataset.pkl"), "rb") as infile:
        self.train_dataset = pickle.load(infile)
      with open(os.path.join(self.pickle_dir, f"en_eval_dataset.pkl"), "rb") as infile:
        self.eval_dataset = pickle.load(infile)
      with open(os.path.join(self.pickle_dir, f"en_test_dataset.pkl"), "rb") as infile:
        self.en_test_dataset = pickle.load(infile)
      with open(os.path.join(self.pickle_dir, f"ch_test_dataset.pkl"), "rb") as infile:
        self.ch_test_dataset = pickle.load(infile)

      en_train_samples = self.train_dataset.samples
      input_samples["en"]["train"] = en_train_samples
      en_eval_samples = self.eval_dataset.samples
      input_samples["en"]["evaluation"] = en_eval_samples

      with open(os.path.join(self.pickle_dir, f"ch_train_dataset.pkl"), "rb") as infile:
        self.train_dataset = pickle.load(infile)
      with open(os.path.join(self.pickle_dir, f"ch_eval_dataset.pkl"), "rb") as infile:
        self.eval_dataset = pickle.load(infile)

      ch_train_samples = self.train_dataset.samples
      input_samples["ch"]["train"] = ch_train_samples
      ch_eval_samples = self.eval_dataset.samples
      input_samples["ch"]["evaluation"] = ch_eval_samples
      en_test_samples = self.en_test_dataset.samples
      input_samples["en"]["test"] = en_test_samples
      ch_test_samples = self.ch_test_dataset.samples
      input_samples["ch"]["test"] = ch_test_samples

      for train_lang in ["en", "ch"]:
        if train_lang != lang:
          eval_sp = 6
        for i in range(8):
          if i == eval_sp:
            continue
          samples[train_lang]["train"].extend(input_samples[train_lang]["train"][1200*i:1200*(i+1)])
          samples[train_lang]["train"].extend(input_samples[train_lang]["evaluation"][80*i:80*(i+1)])
          samples[train_lang]["train"].extend(input_samples[train_lang]["test"][120*i:120*(i+1)])
        samples[train_lang]["evaluation"].extend(input_samples[train_lang]["train"][1200*eval_sp:1200*(eval_sp+1)])
        samples[train_lang]["evaluation"].extend(input_samples[train_lang]["evaluation"][80*eval_sp:80*(eval_sp+1)])
        samples[train_lang]["evaluation"].extend(input_samples[train_lang]["test"][120*eval_sp:120*(eval_sp+1)])
        for i in range(8, 10):
          samples[train_lang]["test"].extend(input_samples[train_lang]["train"][1200*i:1200*(i+1)])
          samples[train_lang]["test"].extend(input_samples[train_lang]["evaluation"][80*i:80*(i+1)])
          samples[train_lang]["test"].extend(input_samples[train_lang]["test"][120*i:120*(i+1)])
        eval_sp = inp_eval_sp
      
      with open(os.path.join(self.pickle_dir, f"{'_'.join(self.LANGS)}_train_dataset.pkl"), "rb") as infile:
        self.train_dataset = pickle.load(infile)
      with open(os.path.join(self.pickle_dir, f"{'_'.join(self.LANGS)}_eval_dataset.pkl"), "rb") as infile:
        self.eval_dataset = pickle.load(infile)

      self.train_dataset.samples = []
      self.eval_dataset.samples = []
      for lang in langs:
        self.train_dataset.samples += samples[lang]["train"]
        self.eval_dataset.samples += samples[lang]["evaluation"]
      self.ch_test_dataset.samples = samples["ch"]["test"]
      self.en_test_dataset.samples = samples["en"]["test"]
      
    else:
      with open(os.path.join(self.pickle_dir, f"{'_'.join(langs)}_train_dataset.pkl"), "rb") as infile:
        self.train_dataset = pickle.load(infile)
      with open(os.path.join(self.pickle_dir, f"{'_'.join(langs)}_eval_dataset.pkl"), "rb") as infile:
        self.eval_dataset = pickle.load(infile)
      with open(os.path.join(self.pickle_dir, f"en_test_dataset.pkl"), "rb") as infile:
        self.en_test_dataset = pickle.load(infile)
      with open(os.path.join(self.pickle_dir, f"ch_test_dataset.pkl"), "rb") as infile:
        self.ch_test_dataset = pickle.load(infile)

    if len(langs) == 2:
      quarter_len = len(self.train_dataset.samples)//4
      self.train_dataset.samples = self.train_dataset.samples[:quarter_len] \
                                + self.train_dataset.samples[2*quarter_len:3*quarter_len] \
                                + self.train_dataset.samples[quarter_len:2*quarter_len] \
                                + self.train_dataset.samples[3*quarter_len:]


  def run_per_split(self, remixed_data=True):
    for lang in self.LANGS:
      for eval_sp in [7, 8]:
        self.sermod = SERwav2vec2(self.PRETRAIN_PATH, self.LEARNING_RATE, self.LINEAR_SIZE)
        
        self.set_data(eval_sp, self.LANGS, lang, remixed_data)
        self.train_model()
        self.test_model()
        for test_lang in ["en", "ch"]:
          self.test_accs[test_lang].append(self.sermod.test_perf[test_lang]["test acc"])
          self.test_losses[test_lang].append(self.sermod.test_perf[test_lang]["test loss"])
        self.output_path = f"wav2vec2_{'_'.join(self.LANGS)}_{self.EPOCHS}ep_eval_sp_{eval_sp}_{lang}_original_data_{not remixed_data}"
        self.perf_path = f"/content/drive/MyDrive/Colab Notebooks/MA LangTech/Thesis/results/performance_{self.output_path}.pkl"
        self.save_model_performance()

    num_its = len(self.test_accs[lang])
    print(f"\n\nTrained on {' '.join(self.LANGS)} Wav2Vec2.0-base voiceprints")
    print(f"Average test performance over {num_its} iteration(s)")
    print("lang accuracy loss")
    print("------------------")
    for lang in ["en", "ch"]:
      avg_acc = sum(self.test_accs[lang])/num_its
      avg_loss = sum(self.test_losses[lang])/num_its
      print(f"{lang:>4} {avg_acc:>8.1f} {avg_loss:>4.1f}")

In [ ]:
epochs = 40
learning_rate = 5e-5
model_type = "mono"
proportion = 1.0

## Train and test models

### Train and test with new split

In [ ]:
if __name__ == "__main__":
  start = time.time()
  
  langs = ["en", "ch"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               model_type=model_type,
                               proportion=proportion)

  pipeline.run_per_split()

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up mono model with
	epochs: 40
	learning rate: 5e-05
	langs: en, ch


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'proj

Begin training


Epoch 1: 100%|██████████| 490/490 [00:03<00:00, 143.12batch/s, accuracy=41.2, loss=1.21]


...

accuracy=76.78571428571429, loss=0.8123715391620281


Epoch 39: 100%|██████████| 490/490 [00:03<00:00, 133.13batch/s, accuracy=91.6, loss=0.212]


accuracy=77.14285714285714, loss=0.7893713902368862


Epoch 40: 100%|██████████| 490/490 [00:03<00:00, 135.45batch/s, accuracy=92.4, loss=0.208]


accuracy=74.35714285714286, loss=0.7949442719371616

Test performance
lang accuracy loss
------------------
  en     73.5  1.5
  ch     76.1  0.8



Trained on en ch Wav2Vec2.0-base voiceprints
Average test performance over 4 iteration(s)
lang accuracy loss
------------------
  en     71.7  1.6
  ch     77.4  0.7

Running the script took 00:12:15


In [ ]:
if __name__ == "__main__":
  start = time.time()
  
  langs = ["en"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               model_type=model_type,
                               proportion=proportion)

  pipeline.run_per_split()

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up mono model with
	epochs: 40
	learning rate: 5e-05
	langs: en


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'proj

Begin training


Epoch 1: 100%|██████████| 490/490 [00:03<00:00, 143.43batch/s, accuracy=39.1, loss=1.27]


accuracy=41.0, loss=1.333104886731931


...

accuracy=69.35714285714286, loss=1.173469066688346


Epoch 39: 100%|██████████| 490/490 [00:03<00:00, 136.26batch/s, accuracy=93.7, loss=0.162]


accuracy=68.35714285714286, loss=1.208323144294138


Epoch 40: 100%|██████████| 490/490 [00:03<00:00, 126.99batch/s, accuracy=93.8, loss=0.153]


accuracy=71.5, loss=1.1322715636832676

Test performance
lang accuracy loss
------------------
  en     72.2  1.7
  ch     54.9  1.6



Trained on en Wav2Vec2.0-base voiceprints
Average test performance over 2 iteration(s)
lang accuracy loss
------------------
  en     71.3  1.7
  ch     58.7  1.5

Running the script took 00:06:02


In [ ]:
if __name__ == "__main__":
  start = time.time()
  
  langs = ["ch"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               model_type=model_type,
                               proportion=proportion)

  pipeline.run_per_split()

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up mono model with
	epochs: 40
	learning rate: 5e-05
	langs: ch


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'proj

Begin training


Epoch 1: 100%|██████████| 490/490 [00:03<00:00, 140.86batch/s, accuracy=49.6, loss=1.05]


accuracy=76.85714285714286, loss=0.7178596242410796


...

accuracy=93.28571428571429, loss=0.17947706367468189


Epoch 39: 100%|██████████| 490/490 [00:03<00:00, 127.11batch/s, accuracy=97.7, loss=0.0634]


accuracy=94.21428571428571, loss=0.17357670766592057


Epoch 40: 100%|██████████| 490/490 [00:03<00:00, 126.98batch/s, accuracy=97.7, loss=0.0594]


accuracy=93.42857142857143, loss=0.1732903886797955

Test performance
lang accuracy loss
------------------
  en     58.8  2.5
  ch     79.9  0.7



Trained on ch Wav2Vec2.0-base voiceprints
Average test performance over 2 iteration(s)
lang accuracy loss
------------------
  en     58.7  2.5
  ch     78.6  0.7

Running the script took 00:05:47


### Train and test with original split

In [ ]:
if __name__ == "__main__":
  start = time.time()
  
  langs = ["en", "ch"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               model_type=model_type,
                               proportion=proportion)

  pipeline.run_per_split(remixed_data=False)

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up mono model with
	epochs: 40
	learning rate: 5e-05
	langs: en, ch


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'proj

Begin training


Epoch 1: 100%|██████████| 600/600 [00:04<00:00, 142.90batch/s, accuracy=43.5, loss=1.18]


accuracy=51.875, loss=0.9901895191892982


...

accuracy=85.5, loss=0.35868812171793657


Epoch 39: 100%|██████████| 600/600 [00:04<00:00, 131.88batch/s, accuracy=92.8, loss=0.191]


accuracy=88.0, loss=0.3444171540093634


Epoch 40: 100%|██████████| 600/600 [00:04<00:00, 129.55batch/s, accuracy=93.1, loss=0.186]


accuracy=86.875, loss=0.3432197355934614

Test performance
lang accuracy loss
------------------
  en     89.3  0.3
  ch     94.2  0.2



Trained on en ch Wav2Vec2.0-base voiceprints
Average test performance over 4 iteration(s)
lang accuracy loss
------------------
  en     88.6  0.3
  ch     94.1  0.2

Running the script took 00:13:19


In [ ]:
if __name__ == "__main__":
  start = time.time()
  
  langs = ["en"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               model_type=model_type,
                               proportion=proportion)

  pipeline.run_per_split(remixed_data=False)

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up mono model with
	epochs: 40
	learning rate: 5e-05
	langs: en


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'proj

Begin training


Epoch 1: 100%|██████████| 600/600 [00:04<00:00, 138.64batch/s, accuracy=40.4, loss=1.24]


accuracy=56.625, loss=1.0145572164468468


...

accuracy=85.125, loss=0.4216840097713355


Epoch 39: 100%|██████████| 600/600 [00:04<00:00, 134.50batch/s, accuracy=93.9, loss=0.161]


accuracy=86.125, loss=0.4142950958740482


Epoch 40: 100%|██████████| 600/600 [00:04<00:00, 135.15batch/s, accuracy=94, loss=0.157]


accuracy=85.375, loss=0.427025406967864

Test performance
lang accuracy loss
------------------
  en     90.8  0.2
  ch     60.1  2.0



Trained on en Wav2Vec2.0-base voiceprints
Average test performance over 2 iteration(s)
lang accuracy loss
------------------
  en     90.3  0.3
  ch     60.4  2.1

Running the script took 00:06:25


In [ ]:
if __name__ == "__main__":
  start = time.time()
  
  langs = ["ch"]

  pipeline = ModelDataPipeline(epochs=epochs,
                               learning_rate=learning_rate,
                               langs=langs,
                               model_type=model_type,
                               proportion=proportion)

  pipeline.run_per_split(remixed_data=False)

  end = time.time()

  print(f"\nRunning the script took {running_time(start, end)}")

Setting up mono model with
	epochs: 40
	learning rate: 5e-05
	langs: ch


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'proj

Begin training


Epoch 1: 100%|██████████| 600/600 [00:04<00:00, 143.44batch/s, accuracy=53.7, loss=0.989]


accuracy=60.375, loss=0.7699186494830065


...

accuracy=92.375, loss=0.23244528640972106


Epoch 39: 100%|██████████| 600/600 [00:04<00:00, 131.16batch/s, accuracy=97.5, loss=0.0664]


accuracy=93.375, loss=0.19740471928921305


Epoch 40: 100%|██████████| 600/600 [00:04<00:00, 140.84batch/s, accuracy=97.6, loss=0.0643]


accuracy=93.875, loss=0.18135514838486

Test performance
lang accuracy loss
------------------
  en     54.4  2.3
  ch     96.7  0.1



Trained on ch Wav2Vec2.0-base voiceprints
Average test performance over 2 iteration(s)
lang accuracy loss
------------------
  en     51.9  2.6
  ch     95.6  0.1

Running the script took 00:06:20


## Transform wav files to wav2vec2.0 pooled VPs

In [ ]:
 # w2v = pipeline.sermod._ser
  # pooler = pipeline.sermod._pooler

  # datasets_to_save = [pipeline.en_test_dataset]

  # for dataset_to_save in datasets_to_save:
  #   for i, (inputs, targets) in enumerate(dataset_to_save.samples):
  #     # inputs, targets = dataset_to_save[i]
  #     if torch.cuda.is_available():
  #       inputs = inputs.cuda()
  #       targets = targets.cuda()
  #     with torch.no_grad():
  #       res = w2v(inputs).last_hidden_state
  #     res = pooler(res.transpose(1,2))
  #     dataset_to_save.samples[i] = (res, targets)

  #   save_langs = '_'.join(dataset_to_save.LANGS)
  #   save_split = dataset_to_save.split
  #   if save_split == "evaluation":
  #     save_split = "eval"
  #   to_save_path = os.path.join(pipeline.pickle_dir, 
  #                               f"{save_langs}_{save_split}_dataset.pkl")
  #   with open(to_save_path, "wb") as outfile:
  #     pickle.dump(dataset_to_save, outfile)

  # i = 0
  # for inputs, targets in DataLoader(pipeline.en_test_dataset, batch_size=1, #pipeline.sermod.BATCH_SIZE, 
  #                                   shuffle=False, num_workers=0, collate_fn=pipeline.sermod.coll_fn):
  #   if torch.cuda.is_available():
  #     inputs = inputs.cuda()
  #     targets = targets.cuda()
  #   # print(inputs.shape, targets.shape, "\n", inputs, targets)
  #   with torch.no_grad():
  #     res = w2v(inputs).last_hidden_state
  #   res = pooler(res.transpose(1,2))
  #   pipeline.en_test_dataset.samples[i] = (res, targets)
  #   i += 1
  # # for i in range(len(pipeline.train_dataset.samples)):
  # #   pipeline.train_dataset.samples[i] = pipeline.train_dataset[i]
  # with open(os.path.join(pipeline.pickle_dir, f"{'_'.join(langs)}_test_dataset.pkl"), "wb") as outfile:
  #   pickle.dump(pipeline.en_test_dataset, outfile)

  # for i in range(len(pipeline.evaluation_dataset.samples)):
  #   pipeline.evaluation_dataset.samples[i] = pipeline.evaluation_dataset[i]
  # with open(os.path.join(pipeline.pickle_dir, f"{'_'.join(langs)}_eval_dataset.pkl"), "wb") as outfile:
  #   pickle.dump(pipeline.eval_dataset, outfile)

  # with open(os.path.join(pipeline.pickle_dir, f"ch_test_dataset_pooled.pkl"), "wb") as outfile:
  #   pickle.dump(pipeline.ch_test_dataset, outfile)